In [1]:
import numpy as np
import os
import time
from vgg16 import VGG16
from keras.preprocessing import image
from imagenet_utils import preprocess_input
from imagenet_utils import decode_predictions
from keras.models import Model
from keras.layers import Dense, Activation, Flatten, merge, Input
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

/home/rana/anaconda3/envs/conaenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/rana/anaconda3/envs/conaenv/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:

count=int(0)

path=os.getcwd()
data_path=path + "/data"
data_dir_list=os.listdir(data_path)

img_data_list=[]

for dataset in data_dir_list:
    images=os.listdir(data_path+"/"+dataset)
    for img in images:
        imgg=image.load_img(data_path+"/"+dataset+"/"+img,target_size=(224,224))
        x=image.img_to_array(imgg)
        x=np.expand_dims(x,axis=0)
        x=preprocess_input(x)
        print("\n input image shape",x.shape,"   count",count," images",img,"\n")
        img_data_list.append(x)
        count+=1
        
        


 input image shape (1, 224, 224, 3)    count 0  images cat.146.jpg 


 input image shape (1, 224, 224, 3)    count 1  images cat.5.jpg 


 input image shape (1, 224, 224, 3)    count 2  images cat.139.jpg 


 input image shape (1, 224, 224, 3)    count 3  images cat.177.jpg 


 input image shape (1, 224, 224, 3)    count 4  images cat.13.jpg 


 input image shape (1, 224, 224, 3)    count 5  images cat.123.jpg 


 input image shape (1, 224, 224, 3)    count 6  images cat.165.jpg 


 input image shape (1, 224, 224, 3)    count 7  images cat.26.jpg 


 input image shape (1, 224, 224, 3)    count 8  images cat.176.jpg 


 input image shape (1, 224, 224, 3)    count 9  images cat.49.jpg 


 input image shape (1, 224, 224, 3)    count 10  images cat.181.jpg 


 input image shape (1, 224, 224, 3)    count 11  images cat.6.jpg 


 input image shape (1, 224, 224, 3)    count 12  images cat.21.jpg 


 input image shape (1, 224, 224, 3)    count 13  images cat.148.jpg 


 input image shape (1, 

In [6]:
img_data_array=np.array(img_data_list)

In [7]:
print(img_data_array.shape)

(808, 1, 224, 224, 3)


In [8]:
img_data=np.rollaxis(img_data_array,1,0)
print(img_data.shape)

(1, 808, 224, 224, 3)


In [9]:
img_data=img_data[0]
print(img_data.shape)

(808, 224, 224, 3)


In [10]:
num_classes = 4
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

In [11]:
labels[0:202]=0
labels[202:404]=1
labels[404:606]=2
labels[606:]=3

In [12]:
names=["cats","dogs","horses","humans"]

In [13]:
Y=np_utils.to_categorical(labels,num_classes)

In [14]:
x,y = shuffle(img_data,Y,random_state=2)

In [15]:
X_train,X_test,Y_train,Y_test=train_test_split(x,y,random_state=2,test_size=0.2)

In [16]:
image_input=Input(shape=(224,224,3))

In [17]:
model=VGG16(weights='imagenet',include_top=True,input_tensor=image_input)

In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [23]:
last_layer=model.get_layer('block5_pool').output

In [24]:
x=Flatten(name='flatten')(last_layer)
x=Dense(128,activation='relu',name='fc1')(x)
x=Dense(128,activation='relu',name='fc2')(x)
out=Dense(num_classes,activation='softmax',name='output')(x)

In [25]:
custom_vgg_model=Model(image_input,out)

In [26]:
custom_vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [28]:
custom_vgg_model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])

In [29]:
for layers in custom_vgg_model.layers[:-1]:
    layers.trainable=False

In [30]:
custom_vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

/home/rana/anaconda3/envs/conaenv/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [ ]:
hist=custom_vgg_model.fit(X_train,Y_train,verbose=1,batch_size=8,epochs=15
                         ,validation_data=(X_test,Y_test))

/home/rana/anaconda3/envs/conaenv/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Train on 646 samples, validate on 162 samples
Epoch 1/15
646/646 [==============================] - 1421s 2s/step - loss: 11.8544 - acc: 0.2601 - val_loss: 12.8348 - val_acc: 0.2037
Epoch 2/15
646/646 [==============================] - 1401s 2s/step - loss: 11.9014 - acc: 0.2616 - val_loss: 12.8348 - val_acc: 0.2037
Epoch 3/15
646/646 [==============================] - 1396s 2s/step - loss: 11.9014 - acc: 0.2616 - val_loss: 12.8348 - val_acc: 0.2037
Epoch 4/15
488/646 [=====================>........] - ETA: 5:19 - loss: 11.7913 - acc: 0.2684